In [ ]:
!pip install mysql-connector-python
# # Colab에 필요한 패키지 설치
!pip install selenium
!pip install webdriver-manager

In [ ]:
# # 2. Chrome 브라우저 설치
!apt-get update  # 패키지 목록 업데이트
!apt-get install -y wget unzip  # wget과 unzip 패키지 설치
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -  # Google의 리눅스 서명 키 추가
!sh -c 'echo "deb [arch=amd64] https://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'  # Chrome 브라우저를 다운로드할 수 있도록 저장소 추가
!apt-get update  # 패키지 목록 다시 업데이트
!apt-get install -y google-chrome-stable  # 최신 안정 버전의 Chrome 브라우저 설치
!apt-get install -y fonts-nanum # 한글폰트 설치

In [3]:
# Selenium과 webdriver-manager를 사용한 Chrome 브라우저 실행 코드
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import bs4
import time
from selenium.webdriver.support.ui import Select
import re
import pandas as pd
# WebDriver 설치 및 브라우저 실행 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-popup-blocking")  # 팝업 차단 옵션
options.add_argument("--headless")  # 헤드리스 모드로 실행
options.add_argument("window-size=1920x1080")  # 헤드리스 모드에서의 창 크기 설정
options.add_argument("--no-sandbox")  # 샌드박스 비활성화
options.add_argument("--disable-dev-shm-usage")  # 메모리 사용 최적화

# user-agent 값 설정
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")

# WebDriver로 Chrome 브라우저 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# 웹페이지 파싱 될때까지 최대 3초 기다림
driver.implicitly_wait(3)

In [4]:
# 브라우저 크기 설정
driver.set_window_size(1920, 1080)  # 원하는 해상도로 설정

In [5]:
#### 합치기

In [6]:
NOPList = []
schoolList = []

In [7]:
#웹사이트 접속
targetUrl = "https://www.schoolinfo.go.kr/"
driver.get(targetUrl)
closeXpath = '//*[@id="pop0000000000000045"]/dl/dd/span/span/div/label/span'
closeXpathEle = driver.find_element(By.XPATH, closeXpath)
closeXpathEle.click()
#키워드 검색
searchXpath = '//*[@id="SEARCH_KEYWORD"]'
driver.find_element(By.XPATH, searchXpath).send_keys("고등학교")
driver.find_element(By.XPATH, searchXpath).send_keys(Keys.ENTER)
time.sleep(2)
searchXpath = '/html/body/div[2]/div[2]/div/div[1]/div/div[2]/form/button'
searchXpathEle = driver.find_element(By.XPATH, searchXpath)
searchXpathEle.click()
time.sleep(2)
#선택 박스
selectXpath = '//*[@id="SIDO_CODE"]'
select_box = driver.find_element(By.XPATH,selectXpath)
select = Select(select_box)
select.select_by_visible_text("경기도")
time.sleep(2)
selectXpath = '//*[@id="GUGUN_CODE"]'
select_box = driver.find_element(By.XPATH,selectXpath)
select = Select(select_box)
select.select_by_visible_text("고양시 일산동구")
time.sleep(2)
searchXpath = '//*[@id="contents"]/div/div[1]/div/a'
searchXpathEle = driver.find_element(By.XPATH, searchXpath)
searchXpathEle.click()
time.sleep(2)
searchXpath = '//*[@id="contents"]/div/div[2]/a'
searchXpathEle = driver.find_element(By.XPATH, searchXpath)
searchXpathEle.click()
time.sleep(4)

In [ ]:
#3페이지
for i in range (0,3):
    #학교 이름 크롤링
    htmlObj = driver.page_source
    bsObj = bs4.BeautifulSoup(htmlObj, "html.parser")
    schoolTag = bsObj.find_all(name = "a" , attrs = {"title" : re.compile("고등학교") , "href" : re.compile("javascript:searchSchul")})
    schoolTag
    for i in range(len(schoolTag)):
        schoolList.append(schoolTag[i].text)
    NOPTagList = bsObj.find_all(name = "div" , attrs = {"class" : "l_box"})
    #인원수
    for i in range(len(NOPTagList)):
        if NOPTagList[i] is None :
            continue;
        NOPTag = NOPTagList[i]
        spanTag = NOPTag.find(name = "span" , attrs = {"class" : "md"})
        if spanTag:
            if len(spanTag.text.split()) >2:
                NOPList.append(spanTag.text.split()[2])
    if i ==2 :
        break
    searchXpath = '//*[@id="contents"]/div/div[3]/div[1]/a[6]'
    searchXpathEle = driver.find_element(By.XPATH, searchXpath)
    searchXpathEle.click()
    time.sleep(4)
    driver.save_screenshot("./here.jpg")

In [9]:
df = pd.DataFrame(zip(schoolList , NOPList) , columns = ["고등학교" , "인원수"])
df

,고등학교,인원수
0,고양국제고등학교,591명
1,백마고등학교,848명
2,백석고등학교,814명
3,백신고등학교,826명
4,세원고등학교,468명
5,안곡고등학교,934명
6,저동고등학교,851명
7,저현고등학교,922명
8,정발고등학교,850명
9,중산고등학교,849명
